In [336]:
import seaborn as sns

cp=sns.color_palette("Set2", n_colors=72).as_hex()
colors={
"RootTask":cp[0],
"DistributionTask":cp[1],
"TrackVeteranTask":cp[2],
"HearingTask":cp[3],
"ScheduleHearingTask":cp[4],
"InformalHearingPresentationTask":cp[5],
"BvaDispatchTask":cp[6],
"EvidenceSubmissionWindowTask":cp[7],
"JudgeDecisionReviewTask":cp[8],
"AttorneyTask":cp[9],
"JudgeAssignTask":cp[10],
"HearingAdminActionVerifyAddressTask":cp[11],
"AssignHearingDispositionTask":cp[12],
"EvidenceOrArgumentMailTask":cp[13],
"TranslationTask":cp[14],
"TranscriptionTask":cp[15],
"QualityReviewTask":cp[16],
"OtherColocatedTask":cp[17],
"AttorneyRewriteTask":cp[18],
"TimedHoldTask":cp[19],
"IhpColocatedTask":cp[20],
"AodMotionMailTask":cp[21],
"ScheduleHearingColocatedTask":cp[22],
"VeteranRecordRequest":cp[23],
"HearingRelatedMailTask":cp[24],
"JudgeDispatchReturnTask":cp[25],
"MissingRecordsColocatedTask":cp[26],
"ReturnedUndeliverableCorrespondenceMailTask":cp[27],
"NoShowHearingTask":cp[28],
"GenericTask":cp[29],
"StayedAppealColocatedTask":cp[30],
"ExtensionRequestMailTask":cp[31],
"ExtensionColocatedTask":cp[32],
"FoiaTask":cp[33],
"PowerOfAttorneyRelatedMailTask":cp[34],
"HearingAdminActionOtherTask":cp[35],
"HearingClarificationColocatedTask":cp[36],
"FoiaColocatedTask":cp[37],
"StatusInquiryMailTask":cp[38],
"PoaClarificationColocatedTask":cp[39],
"HearingAdminActionForeignVeteranCaseTask":cp[40],
"PreRoutingFoiaColocatedTask":cp[41],
"JudgeQualityReviewTask":cp[42],
"CongressionalInterestMailTask":cp[43],
"SpecialCaseMovementTask":cp[44],
"PrivacyActTask":cp[45],
"ChangeHearingDispositionTask":cp[46],
"AddressChangeMailTask":cp[47],
"VacateMotionMailTask":cp[48],
"PreRoutingTranslationColocatedTask":cp[49],
"FoiaRequestMailTask":cp[50],
"AttorneyDispatchReturnTask":cp[51],
"AttorneyQualityReviewTask":cp[52],
"ReconsiderationMotionMailTask":cp[53],
"PrivacyActRequestMailTask":cp[54],
"DeathCertificateMailTask":cp[55],
"AojColocatedTask":cp[56],
"AddressVerificationColocatedTask":cp[57],
"TranslationColocatedTask":cp[58],
"Task":cp[59],
"BoardGrantEffectuationTask":cp[60],
"AppealWithdrawalMailTask":cp[61],
"PendingScanningVbmsColocatedTask":cp[62],
"OtherMotionMailTask":cp[63],
"ControlledCorrespondenceMailTask":cp[64],
"UnaccreditedRepColocatedTask":cp[65],
"PulacCerulloTask":cp[66],
"PreRoutingMissingHearingTranscriptsColocatedTask":cp[67],
"NewRepArgumentsColocatedTask":cp[68],
"MissingHearingTranscriptsColocatedTask":cp[69],
"HearingAdminActionFoiaPrivacyRequestTask":cp[70]
}
customcolors={
"assigned":"#cc6600",
"inprogress":"#006600",
"completed":"#0000cc",
}
colors.update(customcolors)

In [337]:
import json

In [338]:
def rejectCancelled(data):
    return [t for t in data['tasks'] if not t['status']=="cancelled"]

def typesToStringUpTo(i, tcs):
    return ".".join(tcs[0:i+1])

appealIdsLimit=5
appealIdsDict={}
tcsCountsDict={}
tcsSetDict={}
nextlinksDict={}
backlinksDict={}

def flatten(data):
    taskscreationseq=list(map(lambda task: task['type']+"_"+task['assigned_to_type'], rejectCancelled(data)))
    for i in range(len(taskscreationseq)):
        task=taskscreationseq[i]
        if i>0:
            backlinksDict[task]=backlinksDict.get(task, {})
            backlinksDict[task][taskscreationseq[i-1]]=backlinksDict[task].get(taskscreationseq[i-1], 0)+1
        if i+1<len(taskscreationseq):
            nextlinksDict[task]=nextlinksDict.get(task, {})
            nextlinksDict[task][taskscreationseq[i+1]]=nextlinksDict[task].get(taskscreationseq[i+1], 0)+1
        
        typePrefix=typesToStringUpTo(i, taskscreationseq)
        
        tcsSetDict[task]=tcsSetDict.get(task, set())
        tcsSetDict[task].add(typePrefix)
        
        tcsCountsDict[typePrefix]=tcsCountsDict.get(typePrefix,0)+1
        
        appealIdsDict[typePrefix]=appealIdsDict.get(typePrefix, [])
        if(len(appealIdsDict[typePrefix])<appealIdsLimit):
            appealIdsDict[typePrefix].append(data['appeal_id'])
            
        #print(i, tcsCountsDict[typePrefix], appealIdsDict[typePrefix], tcsSetDict[tcs], typePrefix)
    #del data['tasks']
    #return { 'docket': data['docket_type'], 'tcs': taskscreationseq}

In [339]:
def clearData():
    appealIdsDict.clear()
    tcsCountsDict.clear()
    tcsSetDict.clear()
    nextlinksDict.clear()
    backlinksDict.clear()

def loadData(inputfile, dockettype):
    clearData()
    #with open('prepped2.json', 'w') as pf:
    with open(inputfile) as jf:
        for count, line in enumerate(jf):
            data = json.loads(line)
            #removeExtraFields(data)
            if data['docket_type']==dockettype:
                flatdata=flatten(data)
            #print(count, data['appeal_id'], flatdata)
            #pf.write(json.dumps(flatdata)+"\n")


In [340]:
def create_tasklist(basedir):
    with open(f'{basedir}/tasklist.md', 'w') as tlf:
        tlf.write(f'# Task Listing for "{dockettype}" Docket\n\n')
        listing={}
        for taskname,tcsSet in tcsSetDict.items():
            count=sum([tcsCountsDict[tcs] for tcs in tcsSet])
            listing[taskname]=count
        for taskname,count in sorted(listing.items(), key=lambda kv: kv[1], reverse=True):
            tlf.write(f'   * [{taskname}]({taskname}.md) ({count} occurrences)\n')



In [341]:
print(appealIdsDict)            
print(tcsCountsDict)
print(tcsSetDict)

{}
{}
{}


In [342]:
def gen_graphviz(nextlinksDict, backlinksDict, *tasknames):
    edges=set()
    for taskname in tasknames:
        if taskname in nextlinksDict:
            for link,count in sorted(nextlinksDict[taskname].items(), key=lambda kv: kv[1], reverse=True):
                edges.add(f'"{taskname}" -> "{link}" [label={count}]')
        if taskname in backlinksDict:
            for link,count in sorted(backlinksDict[taskname].items(), key=lambda kv: kv[1], reverse=True):
                edges.add(f'"{link}" -> "{taskname}" [label={count}]')
    gstr='digraph G {\nrankdir="LR";\n'
    gstr+="\n".join(edges)
    gstr+="\n}\n"
    return gstr

def save_graphviz(basedir, nextlinksDict, backlinksDict, *tasknames):
    tcsName=abbrev(".".join(tasknames))
    with open(f'{basedir}/dot/{tcsName}.dot', 'w') as gvf:
        gvf.write(gen_graphviz(nextlinksDict, backlinksDict, *tasknames))

print(gen_graphviz(nextlinksDict, backlinksDict, "RootTask_Organization", "DistributionTask_Organization", "EvidenceSubmissionWindowTask_Organization"))
save_graphviz(basedir, nextlinksDict, backlinksDict, "RootTask_Organization", "DistributionTask_Organization", "EvidenceSubmissionWindowTask_Organization")

digraph G {
rankdir="LR";

}



In [343]:
import re
import sys

def find_appeal(inputfile, appeal_id):
    with open(inputfile, "r") as f:
        for count, line in enumerate(f):
            if re.search(f"\"appeal_id\":{appeal_id},", line):
                data = json.loads(line)
                return data
            
appeal_id="41852"
appeal=find_appeal("input2.json", appeal_id)
print(appeal['appeal_id']==int(appeal_id), appeal)

True {'appeal_id': 41852, 'docket_type': 'direct_review', 'updated_at': '2019-11-13 19:13:22 UTC', 'tasks': [{'id': 514013, 'status': 'cancelled', 'type': 'RootTask', 'created_at': '2019-11-12T17:14:08.206Z', 'updated_at': '2019-11-12T17:14:08.877Z', 'assigned_to_type': 'Organization', 'parent_id': None}, {'id': 514014, 'status': 'completed', 'type': 'DistributionTask', 'created_at': '2019-11-12T17:14:08.873Z', 'updated_at': '2019-11-12T18:23:27.474Z', 'assigned_to_type': 'Organization', 'parent_id': 514013}, {'id': 514187, 'status': 'cancelled', 'type': 'JudgeAssignTask', 'created_at': '2019-11-12T18:23:27.463Z', 'updated_at': '2019-11-12T18:23:27.463Z', 'assigned_to_type': 'User', 'parent_id': 514013}]}


In [344]:
def gen_plantuml(appeal, highlighttype="", limit=200):
    pstr = """@startuml
skinparam {
  ObjectBorderColor #555
  ObjectBorderThickness 0
  ObjectFontStyle bold
  ObjectFontSize 14
  ObjectAttributeFontColor #333
  ObjectAttributeFontSize 12
}
"""
    taskId2LabelDict={}
    for task in appeal['tasks']:
        taskLabel=f"{len(taskId2LabelDict)}.{task['type']}"
        taskId2LabelDict[task['id']]=taskLabel
        pstr+=f"  object {taskLabel} {colors[task['type']]}"
        pstr+=" {\n"
        pstr+=task['assigned_to_type']
        if task['type']+"_"+task['assigned_to_type']==highlighttype:
            pstr+=f"  <back:white>    </back>"
        pstr+="\n}\n"
    for task in appeal['tasks']:
        if task['parent_id']:
            pstr+=f"{taskId2LabelDict.get(task['parent_id'])} -- {taskId2LabelDict[task['id']]}\n"
    pstr+="@enduml\n"
    return pstr

print(gen_plantuml(appeal, "DistributionTask_Organization"))

@startuml
skinparam {
  ObjectBorderColor #555
  ObjectBorderThickness 0
  ObjectFontStyle bold
  ObjectFontSize 14
  ObjectAttributeFontColor #333
  ObjectAttributeFontSize 12
}
  object 0.RootTask #66c2a5 {
Organization
}
  object 1.DistributionTask #fc8d62 {
Organization  <back:white>    </back>
}
  object 2.JudgeAssignTask #8da0cb {
User
}
0.RootTask -- 1.DistributionTask
0.RootTask -- 2.JudgeAssignTask
@enduml



In [350]:
def abbrev(tcs):
    return ''.join(filter(lambda x: x.isupper() or x=='.', str(tcs)))

def gen_md_files(inputfile, basedir):
    for taskname,tcsSet in tcsSetDict.items():
        #print(task, tcsSet)
        with open(f'{basedir}/{taskname}.md', 'w') as mdf:
            #mdf.write('# '+taskname.split("_")[0]+" "+taskname.split("_")[1]+'\n\n')
            mdf.write('| [README.md](/README.md) | [Task Listing](tasklist.md) |\n\n')
            mdf.write(f'# {taskname}\n\n')
            mdf.write(f'## Tasks Created Before and After\n\n')
            mdf.write(f"<details><summary>Tasks created before and after {taskname}</summary>\n\n```\n")
            graphviz=gen_graphviz(nextlinksDict, backlinksDict, taskname)
            mdf.write(graphviz)
            mdf.write('```\n</details>\n\n')
            mdf.write(f'![{taskname}](dot/{taskname}.dot.png)\n\n')
            with open(f'{basedir}/dot/{taskname}.dot', 'w') as gvf:
                gvf.write(graphviz)

            mdf.write('**Before:**\n\n')
            if taskname in backlinksDict:
                for link,count in sorted(backlinksDict[taskname].items(), key=lambda kv: kv[1], reverse=True):
                    mdf.write(f"   * [{link}]({link}.md): {count} times\n")
            mdf.write("\n")
            mdf.write('**After:**\n\n')
            if taskname in nextlinksDict:
                for link,count in sorted(nextlinksDict[taskname].items(), key=lambda kv: kv[1], reverse=True):
                    mdf.write(f"   * [{link}]({link}.md): {count} times\n")
            mdf.write("\n")

            mdf.write('## Task Creation Sequences\n\n')
            for tcs in sorted(tcsSet, key=lambda k: tcsCountsDict[k], reverse=True):
                mdf.write(gen_tcs_section(inputfile, basedir, taskname, tcs, tcsCountsDict[tcs], appealIdsDict[tcs]))

def gen_tcs_section(inputfile, basedir, taskname, tcs, count, example_appeal_ids):
    tcsName=abbrev(tcs)
    tstr=f"### {tcsName}\n\n"
    tstr+=f"{count} occurrences (example appeal IDs: {example_appeal_ids})\n\n"
    appealId=appealIdsDict[tcs][0]
    tstr+=f"<details><summary>Task Tree for appeal with ID {appealId}</summary>\n\n```\n"
    appeal=find_appeal(inputfile, appealId)
    plantuml=gen_plantuml(appeal, taskname)
    tstr+=plantuml
    tstr+='```\n</details>\n\n'
    
    tstr+=f'![{tcsName}-{appealId}](uml/{tcsName}-{appealId}.png)\n\n'
    # create associated plantUML file to generate png
    with open(f'{basedir}/uml/{tcsName}-{appealId}.uml', 'w') as umlf:
        umlf.write(plantuml)
        
    return tstr


In [351]:
# 'input2.json' 'input-all.json'
inputfile='input2.json' #'input-all.json'

dockettypes={ "direct_review":"docs-DR", "evidence_submission":"docs-ES", "hearing":"docs-H" }
for dockettype,basedir in dockettypes.items():
    loadData(inputfile, dockettype)

    import os
    os.path.isdir(basedir) or os.mkdir(basedir)
    os.path.isdir(basedir+'/uml') or os.mkdir(basedir+'/uml')
    os.path.isdir(basedir+'/dot') or os.mkdir(basedir+'/dot')

    create_tasklist(basedir)
    gen_md_files(inputfile, basedir)

In [ ]:
tcsSetDict['DistributionTask_Organization']

In [ ]:
print(re.search("[A-Z]","RootTask_Organization.TrackVeteranTask_Organization.n"))
print(dir(""))
print(str(['a', 'b']))
print(abbrev("RootTask_Organization.TrackVeteranTask_Organization.n"))

In [ ]:
typenames={}
with open('input-all.json') as jf:
    for count, line in enumerate(jf):
        line = jf.readline()
        data = json.loads(line)
        for t in data['tasks']:
            count=typenames.get(t['type'],0)
            typenames[t['type']]=count+1

sortednames=sorted(typenames.items(), key = lambda kv:(kv[1], kv[0]))
sortednames.reverse()
i=0
for name,count in sortednames:
    print("\""+name+"\":cp["+str(i)+"],")
    i+=1


In [ ]:
sns.color_palette("Set2", n_colors=72).as_hex()

In [ ]:
for key,color in colors.items():
    print("object "+key+" "+color)